# H2o Meetup SCL Diciembre 2018 - AutoML con H2o
preparado con amor por Spike hola@spikelab.xyz

## Instalación de paquetes e inicialización

Primero debemos instalar ciertos paquetes necesarios. Esto se debe correr solo la primera vez que se usa H2o y cuando se quiera actualizar a una nueva versión.

In [ ]:
!pip install requests
!pip install tabulate
!pip install "colorama>=0.3.8"
!pip install future
!pip uninstall -y h2o #esta línea se asegura de desinstalar versiones anteriores de H2o
!pip install http://h2o-release.s3.amazonaws.com/h2o/rel-wright/10/Python/h2o-3.20.0.10-py2.py3-none-any.whl


Luego, levantamos servidor de H2o dándole la cantidad de memoria adecuada para el tamaño del set de datos. Como regla general, se recomienda disponer memoria equivalente a 4 veces el tamaño del set de datos. Por ejemplo, si el dataset pesa 5GB, entonces debieramos disponer de al menos 20GB de RAM. También se puede limitar el número de cores para h2o en caso de que necesitemos usar otras herramientas.

In [3]:
import h2o
import time
import pandas as pd
from h2o.automl import H2OAutoML #para usar autoML
from h2o.estimators import H2OXGBoostEstimator #para usar XGBoost
from h2o.estimators.kmeans import H2OKMeansEstimator #para clusterizar


param_h2o = { 
        "nthreads":-1           #acá uno puede reemplazar el -1 (=todos los cores disponibles) por otro número
        ,"max_mem_size":'16G'   #cambiar aquí 16G (16 gigabytes de RAM) a memoria disponible para H2o
}

h2o.init(**param_h2o)             #specify max number of bytes. uses all cores by default.

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_191"; OpenJDK Runtime Environment (build 1.8.0_191-8u191-b12-0ubuntu0.18.04.1-b12); OpenJDK 64-Bit Server VM (build 25.191-b12, mixed mode)
  Starting server from /home/ubuntu/miniconda3/envs/spike_basicoV2/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpdaeirrff
  JVM stdout: /tmp/tmpdaeirrff/h2o_ubuntu_started_from_python.out
  JVM stderr: /tmp/tmpdaeirrff/h2o_ubuntu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.20.0.7
H2O cluster version age:,3 months and 20 days !!!
H2O cluster name:,H2O_from_python_ubuntu_h1wmrt
H2O cluster total nodes:,1
H2O cluster free memory:,14.22 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


## Importar datos
Importamos los datos desde una URL o el disco local. Pueden ser archivos CSV o CSV.zip, y también directamente desde S3 (usando S3n://...) o Google Cloud Storage (gs://...). 

Para este ejemplo, usaremos los datos ya integrados de un desafío Kaggle que intenta predecir si una persona caerá en default en un crédito (ver detalles del desafío aquí: https://www.kaggle.com/c/home-credit-default-risk).

In [5]:
#Importar archivo desde AWS:
df = h2o.import_file(path='https://s3.amazonaws.com/spike-meetups/m_train_combined.csv.zip', destination_frame='df')

#Importar desde archivo local:
#df_train = h2o.import_file(path='m_train_combined.csv.zip', destination_frame='df_train')



Parse progress: |█████████████████████████████████████████████████████████| 100%


Miramos los datos y una descripción general del dataframe

In [6]:
df.describe()

Rows:307511
Cols:865




CNT_CHILDREN AMT_INCOME_TOTAL AMT_CREDIT AMT_ANNUITY REGION_POPULATION_RELATIVE DAYS_BIRTH DAYS_EMPLOYED DAYS_REGISTRATION DAYS_ID_PUBLISH OWN_CAR_AGE FLAG_MOBIL FLAG_WORK_PHONE FLAG_CONT_MOBILE FLAG_PHONE FLAG_EMAIL CNT_FAM_MEMBERS REGION_RATING_CLIENT HOUR_APPR_PROCESS_START REG_REGION_NOT_LIVE_REGION REG_REGION_NOT_WORK_REGION LIVE_REGION_NOT_WORK_REGION REG_CITY_NOT_LIVE_CITY REG_CITY_NOT_WORK_CITY LIVE_CITY_NOT_WORK_CITY EXT_SOURCE_1 EXT_SOURCE_2 EXT_SOURCE_3 APARTMENTS_AVG BASEMENTAREA_AVG YEARS_BEGINEXPLUATATION_AVG YEARS_BUILD_AVG COMMONAREA_AVG ELEVATORS_AVG ENTRANCES_AVG FLOORSMAX_AVG FLOORSMIN_AVG LANDAREA_AVG NONLIVINGAPARTMENTS_AVG NONLIVINGAREA_AVG OBS_30_CNT_SOCIAL_CIRCLE DEF_30_CNT_SOCIAL_CIRCLE DEF_60_CNT_SOCIAL_CIRCLE DAYS_LAST_PHONE_CHANGE FLAG_DOCUMENT_2 FLAG_DOCUMENT_3 FLAG_DOCUMENT_4 FLAG_DOCUMENT_5 FLAG_DOCUMENT_6 FLAG_DOCUMENT_7 FLAG_DOCUMENT_8 FLAG_DOCUMENT_9 FLAG_DOCUMENT_10 FLAG_DOCUMENT_11 FLAG_DOCUMENT_12 FLAG_DOCUMENT_13 FLAG_DOCUMENT_14 FLAG_DOCUMENT_15 FLAG_DOCUMENT_16 FLAG_DOCUMENT_17 FLAG_DOCUMENT_18 FLAG_DOCUMENT_19 FLAG_DOCUMENT_20 FLAG_DOCUMENT_21 AMT_REQ_CREDIT_BUREAU_HOUR AMT_REQ_CREDIT_BUREAU_DAY AMT_REQ_CREDIT_BUREAU_WEEK AMT_REQ_CREDIT_BUREAU_MON AMT_REQ_CREDIT_BUREAU_QRT AMT_REQ_CREDIT_BUREAU_YEAR bureau_CREDIT_ACTIVE_Active_count bureau_CREDIT_ACTIVE_Active_count_norm bureau_CREDIT_ACTIVE_Bad debt_count bureau_CREDIT_ACTIVE_Bad debt_count_norm bureau_CREDIT_ACTIVE_Closed_count bureau_CREDIT_ACTIVE_Sold_count bureau_CREDIT_ACTIVE_Sold_count_norm bureau_CREDIT_CURRENCY_currency 1_count_norm bureau_CREDIT_CURRENCY_currency 2_count bureau_CREDIT_CURRENCY_currency 3_count bureau_CREDIT_CURRENCY_currency 3_count_norm bureau_CREDIT_CURRENCY_currency 4_count bureau_CREDIT_CURRENCY_currency 4_count_norm bureau_CREDIT_TYPE_Another type of loan_count bureau_CREDIT_TYPE_Another type of loan_count_norm bureau_CREDIT_TYPE_Car loan_count bureau_CREDIT_TYPE_Car loan_count_norm bureau_CREDIT_TYPE_Cash loan (non-earmarked)_count bureau_CREDIT_TYPE_Cash loan (non-earmarked)_count_norm bureau_CREDIT_TYPE_Consumer credit_count_norm bureau_CREDIT_TYPE_Credit card_count bureau_CREDIT_TYPE_Credit card_count_norm bureau_CREDIT_TYPE_Interbank credit_count bureau_CREDIT_TYPE_Loan for business development_count bureau_CREDIT_TYPE_Loan for business development_count_norm bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_count_norm bureau_CREDIT_TYPE_Loan for the purchase of equipment_count bureau_CREDIT_TYPE_Loan for the purchase of equipment_count_norm bureau_CREDIT_TYPE_Loan for working capital replenishment_count bureau_CREDIT_TYPE_Loan for working capital replenishment_count_norm bureau_CREDIT_TYPE_Microloan_count bureau_CREDIT_TYPE_Microloan_count_norm bureau_CREDIT_TYPE_Mobile operator loan_count bureau_CREDIT_TYPE_Mortgage_count bureau_CREDIT_TYPE_Mortgage_count_norm bureau_CREDIT_TYPE_Real estate loan_count bureau_CREDIT_TYPE_Real estate loan_count_norm bureau_CREDIT_TYPE_Unknown type of loan_count bureau_CREDIT_TYPE_Unknown type of loan_count_norm bureau_DAYS_CREDIT_mean bureau_DAYS_CREDIT_max bureau_DAYS_CREDIT_min bureau_DAYS_CREDIT_sum bureau_CREDIT_DAY_OVERDUE_mean bureau_CREDIT_DAY_OVERDUE_max bureau_CREDIT_DAY_OVERDUE_min bureau_DAYS_CREDIT_ENDDATE_mean bureau_DAYS_CREDIT_ENDDATE_max bureau_DAYS_CREDIT_ENDDATE_min bureau_DAYS_CREDIT_ENDDATE_sum bureau_DAYS_ENDDATE_FACT_mean bureau_DAYS_ENDDATE_FACT_max bureau_DAYS_ENDDATE_FACT_min bureau_AMT_CREDIT_MAX_OVERDUE_mean bureau_CNT_CREDIT_PROLONG_mean bureau_CNT_CREDIT_PROLONG_max bureau_CNT_CREDIT_PROLONG_min bureau_AMT_CREDIT_SUM_mean bureau_AMT_CREDIT_SUM_max bureau_AMT_CREDIT_SUM_min bureau_AMT_CREDIT_SUM_sum bureau_AMT_CREDIT_SUM_DEBT_mean bureau_AMT_CREDIT_SUM_DEBT_max bureau_AMT_CREDIT_SUM_DEBT_min bureau_AMT_CREDIT_SUM_LIMIT_mean bureau_AMT_CREDIT_SUM_LIMIT_max bureau_AMT_CREDIT_SUM_LIMIT_min bureau_AMT_CREDIT_SUM_OVERDUE_mean bureau_AMT_CREDIT_SUM_OVERDUE_max bu

Generamos el dataframe de entrenamiento (df_train) y de validación (df_valid).

In [7]:
df_train, df_valid = df.split_frame([0.8], seed=1234) #80% de training, 20% de validación

## AutoML: Generar una multiplicidad de modelos y elegir el mejor

Elegimos las _features_ que usaremos en la predicción (x), lo que queremos predecir (y) y las _features_ que queremos remover.

In [8]:
x = df_train.columns #features que usaremos para entrenar el modelo y predecir: todas las columnas del dataframe.
df_train['TARGET']=df_train['TARGET'].asfactor()  #paso importante: setear variable a predecir como categórica
y = 'TARGET' #nombre de variable a predecir
x.remove("SK_ID_CURR") #eliminamos ID para que no sea considerado en la predicción

Usamos AutoML, seteando los parámetros de cantidad de tiempo/Número de modelos, métricas de parada, etc.

In [11]:
#aquí debemos setear la cantidad de segundos para la búsqueda de modelos: 3600= 1 hora.
aml = H2OAutoML(max_runtime_secs=3600, seed = 1234, exclude_algos=["GLM", "DeepLearning"], nfolds=5, project_name= "automl_meetup_scl")
aml.train(x = x, y = y, training_frame = df_train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [ ]:
aml.leader.auc(xval=True) 

## XGBoost

Ahora usaremos XGBoost para hacer la misma predicción

In [42]:
#se recomienda jugar con estos parámetros y ver diferencias en los resultados (ej. AUC). Si andan motivados, pueden hacer un grid search.
param = {
      "ntrees" : 400
    , "max_depth" : 5
    , "learn_rate" : 0.1
    , "sample_rate" : 0.7
    , "col_sample_rate_per_tree" : 0.8
    , "min_rows" : 5
    , "seed": 4241
    , "score_tree_interval": 100
    , "nfolds" : 3    
}
model = H2OXGBoostEstimator(**param)
model.train(x = x, y = y, training_frame = df_train, validation_frame=df_valid)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


Revisamos los resultados del modelo.

In [44]:
model

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1544726007196_389


ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.05860083284336553
RMSE: 0.2420760889542078
LogLoss: 0.20881488664874198
Mean Per-Class Error: 0.2240642470450651
AUC: 0.8577443124917659
Gini: 0.7154886249835317
Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.20051894385126692: 


,0,1,Error,Rate
0,266129.0,16557.0,0.0586,(16557.0/282686.0)
1,12916.0,11909.0,0.5203,(12916.0/24825.0)
Total,279045.0,28466.0,0.0958,(29473.0/307511.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.2005189,0.4469423,189.0
max f2,0.1126891,0.5345715,252.0
max f0point5,0.3135129,0.4814020,130.0
max accuracy,0.3788624,0.9274302,102.0
max precision,0.9598812,1.0,0.0
max recall,0.0061442,1.0,395.0
max specificity,0.9598812,1.0,0.0
max absolute_mcc,0.2065700,0.3961649,185.0
max min_per_class_accuracy,0.0919413,0.7757100,272.0
max mean_per_class_accuracy,0.0919413,0.7759358,272.0


Gains/Lift Table: Avg response rate:  8.07 %, avg score:  8.07 %



...

,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error
,2018-12-13 20:25:03,11 min 13.517 sec,0.0,0.5,0.6931472,0.5,1.0,0.9192712
,2018-12-13 20:26:47,12 min 57.014 sec,100.0,0.2532434,0.2308193,0.8058783,7.8768743,0.1283661
,2018-12-13 20:28:04,14 min 14.663 sec,200.0,0.2485916,0.2213036,0.8285717,8.8473890,0.1122789
,2018-12-13 20:29:24,15 min 34.000 sec,300.0,0.2451248,0.2145074,0.8448748,9.3789572,0.1031898
,2018-12-13 20:30:41,16 min 51.304 sec,400.0,0.2420761,0.2088149,0.8577443,9.9427417,0.0958437


Variable Importances: 


variable,relative_importance,scaled_importance,percentage
EXT_SOURCE_2,240.0,1.0,0.0262582
EXT_SOURCE_3,237.0,0.9875,0.0259300
EXT_SOURCE_1,225.0,0.9375,0.0246171
client_installments_AMT_PAYMENT_min_sum,171.0,0.7125,0.0187090
DAYS_BIRTH,150.0,0.625,0.0164114
---,---,---,---
previous_loans_NAME_GOODS_CATEGORY_Furniture_count,1.0,0.0041667,0.0001094
previous_loans_NAME_SELLER_INDUSTRY_Industry_count,1.0,0.0041667,0.0001094
WEEKDAY_APPR_PROCESS_START_WEDNESDAY,1.0,0.0041667,0.0001094
previous_loans_NAME_PAYMENT_TYPE_Cashlessfromtheaccountoftheemployer_count_norm,1.0,0.0041667,0.0001094



See the whole table with table.as_data_frame()


<bound method ModelBase.auc of >

## Clustering usando K-Means

Por último, generaremos un clustering para clasificar a cada uno de las personas en grupos afínes. 

In [10]:
n_inicial=3
n_final=7

results = [H2OKMeansEstimator(model_id="kmeans_"+str(clusters) ,k=clusters, init="Random", seed=1234, standardize=True) for clusters in range(n_inicial,n_final + 1)]
for estimator in results:
    estimator.train(training_frame = df, ignored_columns =['SK_ID_CURR', 'TARGET'])

kmeans Model Build progress: |████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%
kmeans Model Build progress: |████████████████████████████████████████████| 100%


Luego de calcular dónde están los centroides, le asignamos un cluster a cada individuo. Esta operación, muchas veces es lenta y tediosa de hacer, pero con h2o ocurre en segundos.

In [12]:
n=n_inicial

#en este ciclo, pegamos el resultado de cada clustering (con número de centroides de 3 a 7) a cada individuo

for model in results:
    predicted = model.predict(df) #acá asignamos el cluster a cada persona
    df["clustering_"+str(n)+"_centroides"] = (predicted["predict"]+1).asfactor()
    n=n+1

kmeans prediction progress: |█████████████████████████████████████████████| 100%
kmeans prediction progress: |█████████████████████████████████████████████| 100%
kmeans prediction progress: |█████████████████████████████████████████████| 100%
kmeans prediction progress: |█████████████████████████████████████████████| 100%
kmeans prediction progress: |█████████████████████████████████████████████| 100%
